In [19]:
import json

In [20]:
with open("gamemaster.json","r",encoding="utf-8") as file:
    gamemaster=json.load(file)
with open("dict_cp_multiplier.json","r",encoding="utf-8") as file:
    dic=json.load(file)
with open("typechart.json","r",encoding="utf-8") as file:
    typechart=json.load(file)
    
pokemons = gamemaster['pokemon']
moves = gamemaster['moves']

In [21]:
#pokemons

In [22]:
class Pokemon:
    def __init__(self, data):
        self.name = data['speciesName']
        self.baseStats = data['baseStats']
        self.types = data['types']
        self.fastMoves = data['fastMoves']
        self.chargedMoves = data['chargedMoves']

class Player:
    def __init__(self, name):
        self.name = name
        self.pokemon = None
        
    def choose_pokemon(self):
        POKEMON_NAME = input("Choose Your Battle Pokemon: ")
        self.pokemon = Pokemon([item for item in pokemons if item['speciesName'] == POKEMON_NAME][0])
        
class Move:
    def __init__(self, data):
        self.name = data['name']
        self.types = data['type']
        self.power = data['power']
        self.energy = data['energy']
        self.energyGain = data['energyGain']
        self.cooldown = data['cooldown']
        
class Combat:
    def __init__(self, player1, player2):
        self.player = [player1, player2]
        self.stage = 0
        self.dmg_mult = [0, 0]
        self.selected_fast_move = [None, None]
        self.selected_charged_move = [None, None]
        self.energy = [0, 0]
        self.fastMove_cd = [0, 0]
        self.chargedMove_cd = [0, 0]
        self.hp = [player1.pokemon.baseStats['hp'], player2.pokemon.baseStats['hp']]
        
        
    def info(self):
        """Check Game Status"""
        for i in range(2):
            print("Player{}: {}\nPokemon: {}\nDamage: {}\nSelected Skill: {}\nSelected Ultimate: {}\n".format(i + 1, 
                  self.player[i].name, self.player[i].pokemon.name, self.dmg_mult[i], 
                  self.selected_fast_move[i].name, self.selected_charged_move[i]))

    def check_type(self,typechart):
        """Check Counters"""
        self.dmg_mult = [1, 1]
        for p1_type in self.player[0].pokemon.types:
            if p1_type != 'none':
                for p2_type in self.player[1].pokemon.types:
                    if p2_type != 'none':
                        if p1_type in typechart[p2_type]['traits']['weaknesses']:
                            self.dmg_mult[0] = self.dmg_mult[0] * 1.6
                            print('Player1 counter Player2')
                        if p1_type in typechart[p2_type]['traits']['resistances']:
                            self.dmg_mult[0] = self.dmg_mult[0] * 0.625
                            print('player2 resist Player1')
                        if p1_type in typechart[p2_type]['traits']['immunities']:
                            self.dmg_mult[0] = 0
                            print('Player2 immunize Player1')
        
        for p2_type in self.player[1].pokemon.types:
            if p2_type != 'none':
                for p1_type in self.player[0].pokemon.types:
                    if p1_type != 'none':
                        if p2_type in typechart[p1_type]['traits']['weaknesses']:
                            self.dmg_mult[1] = self.dmg_mult[1] * 1.6
                            print('Player2 counter Player1')
                        if p2_type in typechart[p1_type]['traits']['resistances']:
                            self.dmg_mult[1] = self.dmg_mult[1] * 0.625
                            print('Player1 resist Player2')
                        if p2_type in typechart[p1_type]['traits']['immunities']:
                            self.dmg_mult[1] = 0
                            print('Player1 immunize Player2')
                            
        print("Player1's damage is {}, Player2's damage is {}".format(self.dmg_mult[0],self.dmg_mult[1]))
        
    def select_fast_move(self, player_num):
        """Player select skill"""
        print("{}'s skills' : {}".format(self.player[player_num - 1].pokemon.name, 
                                      self.player[player_num - 1].pokemon.fastMoves))
        while True:
            selected_fast_move = input("Select the skill in this game: ")
            if selected_fast_move in self.player[player_num - 1].pokemon.fastMoves:
               break 
            print("Error: Please select the skill in {}".format(self.player[player_num - 1].pokemon.name))
            
        self.selected_fast_move[player_num - 1] = Move([move for move in moves if move['moveId'] == selected_fast_move][0])
        
            
    def select_charged_move(self, player_num):
        """Player select ultimate"""
        print("{}'s ultimate': {}".format(self.player[player_num - 1].pokemon.name, 
                                      self.player[player_num - 1].pokemon.chargedMoves))
        while True:
            selected_charged_move = input("Select the ultimate in this round: ")
            if selected_charged_move in self.player[player_num - 1].pokemon.chargedMoves:
               break 
            print("Error: Please select the skill in {}".format(self.player[player_num - 1].pokemon.name))
            
        self.selected_charged_move[player_num - 1] = Move([move for move in moves if move['moveId'] == selected_charged_move][0])    
        
    def attack(self, attacker, move):
        atkplus = 1
        if move.types in self.player[attacker].pokemon.types:
            atkplus = 1.25
        atk_divide_deff = self.player[attacker].pokemon.baseStats['atk'] / self.player[1 - attacker].pokemon.baseStats['def']
        damage = 1 + round(0.5 * move.power * atk_divide_deff * atkplus * self.dmg_mult[attacker])
        self.hp[1 - attacker] = self.hp[1 - attacker] - damage
        if move.energy == 0:
            self.fastMove_cd[attacker] = self.fastMove_cd[attacker] + move.cooldown
            self.energy[attacker] = self.energy[attacker] + move.energyGain
        else:
            self.chargedMove_cd[attacker] = self.chargedMove_cd[attacker] + move.cooldown
            self.energy[attacker] = self.energy[attacker] - move.energy
    
    def start(self):
        while self.hp[0] > 0 and self.hp[1] > 0:
            self.stage = self.stage + 1
            print("Round {}".format(self.stage))
            for player_num in range(2):
                if self.energy[player_num] >= self.selected_charged_move[player_num].energy and self.chargedMove_cd[player_num] == 0:
                    self.attack(player_num, self.selected_charged_move[player_num])
                    print("Player {}'s {} used ultimate".format(player_num + 1, self.player[player_num].pokemon.name))
                elif self.fastMove_cd[player_num] == 0:
                    self.attack(player_num, self.selected_fast_move[player_num])
                    print("Player {}'s {} used skill".format(player_num + 1, self.player[player_num].pokemon.name))
                else:
                    print("Player {}'s {} is in cooldown".format(player_num + 1, self.player[player_num].pokemon.name))
            
            print("\n")
            
            for player_num in range(2):
                self.fastMove_cd[player_num] = max(self.fastMove_cd[player_num] - 1000, 0)
                self.chargedMove_cd[player_num] = max(self.chargedMove_cd[player_num] - 1000, 0)
    
            if self.hp[0] <= 0 and self.hp[1] > 0:
                print("Player2 win")
            elif self.hp[1] <= 0 and self.hp[0] > 0:
                print("Player1 win")
            elif self.hp[1] <= 0 and self.hp[0] <= 0:
                print("Draw")
            else:
                for i in range(2):
                    print("Player {}'s {}:\nHP: {}/{}\nenergy: {}\nskill cooldown: {}\nutlimate cooldown: {}\n".format(i + 1, self.player[i].pokemon.name, 
                                                                            self.hp[i], self.player[i].pokemon.baseStats['hp'],
                                                                            self.energy[i], self.fastMove_cd[i], self.chargedMove_cd[i]))
            
        
        
        
        
        

In [23]:
p1 = Player('Julius')
p2 = Player('Mingtsung')

In [24]:
p1.choose_pokemon()
p2.choose_pokemon()

Choose Your Battle Pokemon: Bulbasaur
Choose Your Battle Pokemon: Pikachu


In [25]:
combat = Combat(p1,p2)

In [26]:
combat.check_type(typechart)

Player1 resist Player2
Player1's damage is 1, Player2's damage is 0.625


In [27]:
combat.select_fast_move(1)
combat.select_fast_move(2)
combat.select_charged_move(1)
combat.select_charged_move(2)

Bulbasaur's skills' : ['TACKLE', 'VINE_WHIP']
Select the skill in this game: VINE_WHIP
Pikachu's skills' : ['PRESENT', 'QUICK_ATTACK', 'THUNDER_SHOCK']
Select the skill in this game: THUNDER_SHOCK
Bulbasaur's ultimate': ['POWER_WHIP', 'SEED_BOMB', 'SLUDGE_BOMB']
Select the ultimate in this round: SLUDGE_BOMB
Pikachu's ultimate': ['DISCHARGE', 'SURF', 'THUNDER', 'THUNDERBOLT', 'WILD_CHARGE']
Select the ultimate in this round: THUNDERBOLT


In [28]:
combat.start()

Round 1
Player 1's Bulbasaur used skill
Player 2's Pikachu used skill


Player 1's Bulbasaur:
HP: 126/128
energy: 8
skill cooldown: 0
utlimate cooldown: 0

Player 2's Pikachu:
HP: 106/111
energy: 9
skill cooldown: 0
utlimate cooldown: 0

Round 2
Player 1's Bulbasaur used skill
Player 2's Pikachu used skill


Player 1's Bulbasaur:
HP: 124/128
energy: 16
skill cooldown: 0
utlimate cooldown: 0

Player 2's Pikachu:
HP: 101/111
energy: 18
skill cooldown: 0
utlimate cooldown: 0

Round 3
Player 1's Bulbasaur used skill
Player 2's Pikachu used skill


Player 1's Bulbasaur:
HP: 122/128
energy: 24
skill cooldown: 0
utlimate cooldown: 0

Player 2's Pikachu:
HP: 96/111
energy: 27
skill cooldown: 0
utlimate cooldown: 0

Round 4
Player 1's Bulbasaur used skill
Player 2's Pikachu used skill


Player 1's Bulbasaur:
HP: 120/128
energy: 32
skill cooldown: 0
utlimate cooldown: 0

Player 2's Pikachu:
HP: 91/111
energy: 36
skill cooldown: 0
utlimate cooldown: 0

Round 5
Player 1's Bulbasaur used skill
Play